# TA Signal Finder

The combined script for TA signals. At the moment ATH, x-year max and price-ma crossing are covered. All signals, like maxima and ATHs, are calculated based only on close prices. What can be found as not fully correct, but it is done on purpose.

In [56]:
import pandas as pd
from yahoofinancials import YahooFinancials
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os
import sys

In [80]:
class SignalFinder:
    ''' The combined script for TA signals. At the moment ATH, x-year max and price-ma crossing are covered.
        The supported indexes are wig20, mwig40 and swig80. That will be allowed to use combinations with '+' separator
        e.g. 'wig20+mwig40'.
        At the moment only wig20+mwig40+swig80 was fully tested. '''

    def __init__(self, index):
        self.index = index
        self.file_name_part = index.replace('+', '_')
        self.tickers = []
        self.today_price_dict = {}
        self.today_ma_dict = {}
        self.new_ath = []
        self.new_max = []
        self.interval = 'daily'
        self.aths_for_older = {}
        self.actual_ma = 0

    def get_new_ath_df(self, save=False):
        ''' Get new ath data frame for the components of provided index.
            Start day is statically set to the first date of data occurrence.
            End day is today, but yahoo return historical data up to last session.
            Interval is always 1D. '''
        if not self.tickers:
            self.tickers = self.get_tickers_list()
        start_date = '1971-01-01' # for wig there are data since 2000 anyway
        end_date = datetime.today().strftime('%Y-%m-%d')
        data = self._get_historical_data(start_date, end_date)
        close_df = self._get_close_df(data)
        ath_ser = close_df.max()
        self.__set_old_aths()
        ath_ser = self._update_ath_ser_by_dict(ath_ser, self.aths_for_older)
        ath_df = ath_ser.to_frame()
        if save:
            self.__save_initial_ath_df(ath_df)
        return ath_df

    def get_tickers_list(self):
        ''' Get the ticker list. Currently only wig20, mwig40 and swig80 are supported. '''
        if self.tickers:
            return self.tickers
        else:
            index_list = self._get_index_list()
            for index in index_list:
                self.tickers += self._get_wig_tickers(index)
            return self.tickers

    def _get_index_list(self):
        index_list = self.index.split('+')
        return index_list

    def _get_wig_tickers(self, wig):
        ''' Get WIG components list. Only the wig20, mwig40 and swig80 are supported.
            Returns a list of tickers'''
        parsed_content = self._parse_wig_comps_data(wig=wig)
        tickers = [ticker.get_text()[1:] + '.WA' for ticker in parsed_content]
        return tickers

    def _parse_wig_comps_data(self, wig):
        ''' Parse data from https://strefainwestorow.pl/notowania/gpw/{wig}/komponenty
            Returns the pre-processed table with the tickers.'''
        sublink = self._get_wig_sublink(wig)
        source = requests.get(f'https://strefainwestorow.pl/notowania/gpw/{sublink}/komponenty').text
        soup = BeautifulSoup(source, 'lxml')
        table_w_symbols = soup.find_all('a', class_="instrument-symbol")
        return table_w_symbols

    def _get_wig_sublink(self, wig):
        if wig == 'wig20':
            sublink = 'wig20-wig20'
        elif wig == 'mwig40':
            sublink = 'mwig40-mwig40'
        elif wig == 'swig80':
            sublink = 'swig80-swig80'
        return sublink

    def _get_historical_data(self, start_date, end_date):
        yahoo_financials = YahooFinancials(self.tickers)
        stats = yahoo_financials.get_historical_price_data(start_date, end_date, time_interval=self.interval)
        return stats

    @staticmethod
    def _get_close_df(dist_data):
        ''' Process the historical price data fetched from Yahoo.
            Returns data frame with close prices of all tickers. '''
        df_f = pd.DataFrame()
        for ticker, hist_data in dist_data.items():
            if 'prices' in hist_data.keys():
                df = pd.DataFrame(hist_data['prices'], columns=['close', 'formatted_date'])
                df.set_index('formatted_date', inplace=True)
                df.index = pd.to_datetime(df.index)
                df.rename(columns={'close': f'{ticker}'}, inplace=True)
                df_f = pd.concat([df_f, df], axis=1)
        return df_f

    def __set_old_aths(self):
        ''' Manually assigned aths for current (2022Q2) WIG20, mWIG40 and sWIG80 components.
            Every change of the indexes components the list need to be updated.
            Dict has to contain only keys which are present in index components. '''
        aths_for_older = {
            'ACP.WA': 70.00,
            'CDR.WA': 125.5,
            'KGH.WA': 9.95,
            'MBK.WA': 121.3,
            'OPL.WA': 13.8,
            'PEO.WA': 33.15,
            'PKN.WA': 18.09,
            'BDX.WA': 36.87,
            'CMR.WA': 56,
            'BHW.WA': 29.7,
            'ING.WA': 26.94,
            'KTY.WA': 35,
            'MIL.WA': 9.93,
            'AGO.WA': 42.77,
            'AMC.WA': 52.2,
            'BRS.WA': 0.28,
            'BOS.WA': 74.53,
            'ECH.WA': 0.28,
            'FTE.WA': 12.56,
            'RFK.WA': 13.55,
            'SNK.WA': 3.79,
            'STX.WA': 37.09,
            'VRG.WA': 7.41
        }
        self.aths_for_older = {k: aths_for_older[k] for k in self.tickers if k in aths_for_older}

    @staticmethod
    def _update_ath_ser_by_dict(ser_in, ath_dict):
        ''' Update ath series based on the new prices stored in dictionary.
            Returns new series. '''
        ser = ser_in.copy()
        print(ser)
        mask = ser.loc[ath_dict.keys()] > list(ath_dict.values())
        index_to_replace = list(mask[mask == False].index)
        values_to_update = [ath_dict[k] for k in index_to_replace]
        ser.loc[index_to_replace] = values_to_update
        return ser

    def __save_initial_ath_df(self, df):
        ''' Save initial ATH data frame. '''
        today = datetime.today().strftime('%Y-%m-%d')
        df.index.name = 'Ticker'
        df.rename(columns={0: "ATH"}, inplace=True)
        df.to_csv(f'ATH_Data/ATH_{self.file_name_part}_{today}.csv')

    def get_tickers_list_from_file(self):
        ''' Can be use to set tickers list if you are sure there was no changes in components list. '''
        with open(f"Tickers/{self.file_name_part}_tickers.txt", "r") as file:
            self.tickers = file.read().splitlines()
        return self.tickers

    def save_tickers_list(self):
        ''' Save tickers list to file if the they were fetched.
            The files are stored in Tickers directory. '''
        if self.tickers:
            with open(f"Tickers/{self.file_name_part}_tickers.txt", "w") as tickers:
                tickers.write('\n'.join(self.tickers))
        else:
            print('There is no tickers in tickers list')

    def _check_components_validity(self):
        ''' To check if there were changes in indexes which indicate the ATH data frame update. '''
        if self.tickers:
            return self._compare_lists(self._read_tickers_list(), self.tickers)
        else:
            self.get_tickers_list()
            return self._compare_lists(self._read_tickers_list(), self.tickers)

    def _read_tickers_list(self):
        ''' Return the tickers list from file located in Tickers directory
            without setting the initialized ticker variable. '''
        try:
            with open(f"Tickers/{self.file_name_part}_tickers.txt", "r") as file:
                tickers_file = file.read().splitlines()
            return tickers_file
        except FileNotFoundError:
            print('There is no such file in Tickers directory. Please save the ticker list using save_tickers_list.')
            sys.tracebacklimit=0
            raise

    @staticmethod
    def _compare_lists(list1, list2):
        return list1 == list2

    def get_comp_ath_list(self):
        ''' Get component/s with new ATH/s. Before running this function the tickers list
         should be set and compered with previous one.
         The return is a list of tickers. '''
        cur_ath_df = self._read_csv_data('ath')
        if not self.today_price_dict:
            self._get_current_prices_dict()
        mask = cur_ath_df.loc[self.today_price_dict.keys(), 'ATH'] < list(self.today_price_dict.values())
        self.new_ath = list(cur_ath_df.loc[self.today_price_dict.keys()][mask].index)
        return self.new_ath

    def _read_csv_data(self, signal):
        ''' The supported signals so far are 'ath', 'ma', 'x-years'
            The path is the directory to search.
            The file_path is a file path of the latest csv file.
            Returns the data frame of file_path. '''
        try:
            path = self._get_dir_by_signal(signal)
            file_path = self._get_last_saved_csv(path)
            return pd.read_csv(file_path, index_col='Ticker')
        except ValueError:
            print('Could be there is no file for such index. '
                  'Please run get_new_ath_df(save=True) function first.')
            sys.tracebacklimit=0
            raise

    @staticmethod
    def _get_dir_by_signal(signal):
        if signal == 'ath':
            return 'ATH_Data'
        elif signal == 'ma':
            return 'MA_Cross_Data'
        elif signal == 'x-years':
            return 'X-years_Max_Data'

    def _get_last_saved_csv(self, path):
        files = os.listdir(path)
        files_paths = [os.path.join(path, basename) for basename in files]
        return self._get_last_csv_path(files_paths)

    def _get_last_csv_path(self, paths):
        ''' Search for latest .csv file with appropriate file name (file_name_part) in the paths table.
            Returns the last file path '''
        last_file = max(paths, key=os.path.getctime)
        if last_file[-4:] == '.csv' and self.__check_file_vs_index_name(last_file):
            return last_file
        else:
            paths.remove(last_file)
            return self._get_last_csv_path(paths)

    def __check_file_vs_index_name(self, path):
        index_name = path.split('_')[2:-1] # suits ATH_Data only
        index_name = '_'.join(index_name)
        return True if self.file_name_part == index_name else False

    def _get_current_prices_dict(self):
        yahoo_financials = YahooFinancials(self.tickers)
        self.today_price_dict = yahoo_financials.get_current_price()
        return self.today_price_dict

    def save_updated_ath_df(self):
        ''' Save new ATH data frame if get_comp_ath_list returned some ticker/s. '''
        if self.new_ath:
            today = datetime.today().strftime('%Y-%m-%d')
            new_ath_df = self._update_ath_df_by_dict()
            new_ath_df.to_csv(f'ATH_Data/ATH_{self.file_name_part}_{today}.csv')
        else:
            print('There is no need to save updated ATH csv because of none new ATH.')

    def _update_ath_df_by_dict(self):
        ''' Update ath data frame based on the new prices stored in dictionary.
            Returns new data frame. '''
        df = self._read_csv_data('ath')
        mask = df.loc[self.today_price_dict.keys(), 'ATH'] < list(self.today_price_dict.values())
        index_to_replace = list(mask[mask].index)
        values_to_update = [self.today_price_dict[k] for k in index_to_replace]
        df.loc[index_to_replace] = values_to_update
        return df

    def get_comp_x_years_max(self, years):
        ''' Return the ticker/s name/s which beat its/theirs the x-years max/s.
            The log file in X-years_Max_Data/ is automatically updated. '''
        # to add the read/get tickers list?
        start_date, end_date = self._get_start_end_date(years)
        data = self._get_historical_data(start_date, end_date)
        close_df = self._get_close_df(data)
        max_ser = close_df.max()
        if not self.today_price_dict:
            self._get_current_prices_dict()
        self.new_max = self._get_new_maxs(max_ser)
        # check if x-years max log exist, if not create template
        if not self.__check_if_file_exist(f'X-years_Max_Data/{self.file_name_part}_x_years_max.txt'):
            self.__create_x_years_max_log_temp()
        self._update_x_years_max_log_file(max_ser, years)
        return self.new_max

    @staticmethod
    def _get_start_end_date(years):
        end = datetime.today() # - timedelta(days=1) <-- not needed because Yahoo already return [start, today-1] sessions
        start = end - relativedelta(years=years)
        return start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')

    def _get_new_maxs(self, cur_max_ser):
        ''' Returns tickers with new x-year maxes based on series with last maxima and new prices dictionary. '''
        mask = cur_max_ser.loc[self.today_price_dict.keys()] < list(self.today_price_dict.values())
        return list(cur_max_ser.loc[self.today_price_dict.keys()][mask].index)

    @staticmethod
    def __check_if_file_exist(path):
        return os.path.isfile(path)
    
    def __create_x_years_max_log_temp(self):
        with open(f'X-years_Max_Data/{self.file_name_part}_x_years_max.txt', 'a') as file:
            file.write('Date, Ticker, Years, Old_max, New_max\n')
    
    def _update_x_years_max_log_file(self, cur_max_ser, years):
        ''' Update the log file with all new x-years maxima. '''
        today = datetime.today().strftime('%Y-%m-%d')
        with open(f'X-years_Max_Data/{self.file_name_part}_x_years_max.txt', 'a') as file:
            for ticker in self.new_max:
                file.write(
                    f'{today}, {ticker}, {years}, {self._round_2(cur_max_ser[ticker])}, {self._round_2(self.today_price_dict[ticker])}\n')
    
    @staticmethod
    def _round_2(x):
        return round(x, 2)

    def get_ma_position_change(self, ma):
        ''' Get (print) list of changed price positions relative to the MA value.
            The file logs are saved after all actions.
            That can be run only once per ma. Otherwise the outcome can by all equal positions.
            None is returned. '''
        if not self.today_price_dict:
            self._get_current_prices_dict()
        if not self.today_ma_dict or self.actual_ma != ma:  # actually self.actual_ma != ma should be enough
            self.actual_ma = ma
            self._get_moving_avg_dict(ma)
        ma_pos_list = self._get_ma_position_list()
        price_ma_df = self._create_ma_position_df(ma_pos_list)
        last_price_ma_df = self._read_last_price_ma_pos_df(ma)
        tickers_ser = self._get_ma_position_change_tickers(last_price_ma_df, price_ma_df)
        self.__print_ma_positon_change(last_price_ma_df, price_ma_df, tickers_ser)
        change_log_df = self._read_ma_pos_change_log(ma)
        change_log_df = self._fill_in_ma_pos_change_log(change_log_df, last_price_ma_df, price_ma_df, tickers_ser)
        self.__save_ma_log_and_last_price_ma_dfs(price_ma_df, change_log_df, ma)

    def _get_moving_avg_dict(self, ma):
        ''' Get today ma values. At the moment only ma 50 and 200 are supported by yahoo finance.
            Returns a dictionary with tickers and ma values for each. '''
        if not self.today_ma_dict:
            yahoo_financials = YahooFinancials(self.tickers)
            if ma == 50:
                self.today_ma_dict = yahoo_financials.get_50day_moving_avg()
            elif ma == 200:
                self.today_ma_dict = yahoo_financials.get_200day_moving_avg()
            else:
                print(f'Wrong ma value {ma}. Only 50 and 200 are supported')
        return self.today_ma_dict

    def _get_ma_position_list(self, price_dict=None, ma_dict=None):
        ''' By default the self.today_price_dict is used.
            The self.today_ma_dict is fixed by _get_moving_avg_dict function.
            Returns the list of price positions vs MA value. '''
        if not price_dict:
            price_dict = self.today_price_dict
        if not ma_dict:
            ma_dict = self.today_ma_dict
        ma_position_list = []
        for ticker, price in price_dict.items():
            ma_value = ma_dict[ticker]
            if price > ma_value:
                ma_position_list.append('above')
            elif price < ma_value:
                ma_position_list.append('below')
            else:
                ma_position_list.append('equal')
        return ma_position_list

    def _create_ma_position_df(self, ma_position_list, price_dict=None):
        ''' By default the self.today_price_dict is used.
            The self.today_ma_dict is fixed by _get_moving_avg_dict function.
            The ma_position_list needs to be provided.
            Returns the data frame containing Ticker, Price, MA value and Position (Price vs MA) data. '''
        if not price_dict:
            price_dict = self.today_price_dict
        price_ma_df = pd.DataFrame(columns=['Ticker', 'Price', 'MA', 'Position'])
        price_ma_df['Ticker'] = price_dict.keys()
        price_ma_df['Price'] = price_dict.values()
        price_ma_df['MA'] = self.today_ma_dict.values()
        price_ma_df['Position'] = ma_position_list
        return price_ma_df

    def _read_last_price_ma_pos_df(self, ma):
        try:
            return pd.read_csv(f'MA_Cross_Data/{self.file_name_part}_price_ma{ma}_position.csv')
        except FileNotFoundError:
            print('There is no csv file to load. Please run get_initial_price_ma_position function first.')
            sys.tracebacklimit=0
            raise

    @staticmethod
    def _get_ma_position_change_tickers(last_price_ma_df, price_ma_df):
        ''' Returns the series of tickers for which price vs ma value position changed. '''
        mask = (last_price_ma_df['Position'] != price_ma_df['Position'])
        tickers_ser = price_ma_df[mask]['Ticker']
        #         self.__print_ma_positon_change(last_price_ma_df, price_ma_df, tickers_ser)
        return tickers_ser

    @staticmethod
    def __print_ma_positon_change(last_price_ma_df, price_ma_df, tickers_ser):
        ''' Print the information of price vs ma value position changes. '''
        for i in tickers_ser.index:
            if (last_price_ma_df.loc[i]['Position'] == 'above') & (price_ma_df.loc[i]['Position'] == 'below'):
                print(price_ma_df.loc[i]['Ticker'], 'LOST the MA')
            elif (last_price_ma_df.loc[i]['Position'] == 'above') & (price_ma_df.loc[i]['Position'] == 'equal'):
                print(price_ma_df.loc[i]['Ticker'], 'DROPPED perfectly on the MA')
            elif (last_price_ma_df.loc[i]['Position'] == 'below') & (price_ma_df.loc[i]['Position'] == 'above'):
                print(price_ma_df.loc[i]['Ticker'], 'BEAT the MA')
            elif (last_price_ma_df.loc[i]['Position'] == 'below') & (price_ma_df.loc[i]['Position'] == 'equal'):
                print(price_ma_df.loc[i]['Ticker'], 'REACHED perfectly the MA')
            elif (last_price_ma_df.loc[i]['Position'] == 'equal') & (price_ma_df.loc[i]['Position'] == 'above'):
                print(price_ma_df.loc[i]['Ticker'], 'BEAT (from prev. eq) the MA')
            elif (last_price_ma_df.loc[i]['Position'] == 'equal') & (price_ma_df.loc[i]['Position'] == 'below'):
                print(price_ma_df.loc[i]['Ticker'], 'LOST (from prev. eq) the MA')
        if tickers_ser.empty:
            print('It seems there is no changes. Is it weekend, holiday or market was so boring today?')

    def _read_ma_pos_change_log(self, ma):
        try:
            return pd.read_csv(f'MA_Cross_Data/{self.file_name_part}_price_ma{ma}_change_log.csv', index_col='Index')
        except FileNotFoundError:
            print('There is no change log. Going to create and return blank one.')
            return self._create_new_ma_change_log(ma)

    @staticmethod
    def _create_new_ma_change_log(ma):
        df = pd.DataFrame(
            columns=['Index', 'Date', 'Ticker', 'Prev. Price', 'Curr. Price', 'Prev. MA', 'Curr. MA', 'Prev. Position',
                     'Curr. Position'])
        df.set_index('Index', inplace=True)
        return df

    @staticmethod
    def _fill_in_ma_pos_change_log(change_log_df, last_price_ma_df, price_ma_df, tickers_ser):
        ''' Goes through all tickers for which the price vs ma position changed and fill the change log.
            Returns new change log data frame. '''
        today = datetime.today().strftime('%Y-%m-%d')
        for idx in tickers_ser.index:
            change_log_df.loc[len(change_log_df)] = [today, price_ma_df.loc[idx]['Ticker'],
                                                     last_price_ma_df.loc[idx]['Price'], price_ma_df.loc[idx]['Price'],
                                                     last_price_ma_df.loc[idx]['MA'], price_ma_df.loc[idx]['MA'],
                                                     last_price_ma_df.loc[idx]['Position'],
                                                     price_ma_df.loc[idx]['Position']]
        return change_log_df

    def __save_ma_log_and_last_price_ma_dfs(self, price_ma_df, change_log_df, ma):
        self.__save_price_ma_df(price_ma_df, ma)
        self.__save_change_log_df(change_log_df, ma)

    def __save_change_log_df(self, change_log_df, ma):
        change_log_df.to_csv(f'MA_Cross_Data/{self.file_name_part}_price_ma{ma}_change_log.csv')

    def __save_price_ma_df(self, price_ma_df, ma):
        price_ma_df.to_csv(f'MA_Cross_Data/{self.file_name_part}_price_ma{ma}_position.csv')

    def get_initial_price_ma_position(self, ma):
        ''' Get initial price vs ma value data frame.
            The data for last session are fetched.
            The ma value for last session is calculated from historical close prices.
            The .csv file with price, ma and position is saved in MA_Cross_Data directory.
            Returns the saved data frame. '''
        start_date, end_date = self._get_last_session_day(), datetime.today().strftime('%Y-%m-%d')
        last_session_price_data = self._get_historical_data(start_date, end_date)
        last_session_price_dict = self._get_price_dict_from_hist_data(last_session_price_data)
        # calculate ma value for last session
        # 1.7*ma was chosen arbitrary to make sure we have ma sessions in the output
        # 5 sessions of 7 days gives 1.4 but let's have some margin for holidays etc
        start_date, end_date = (datetime.today() - timedelta(days=1.7*ma)).strftime('%Y-%m-%d'), datetime.today().strftime('%Y-%m-%d')
        last_ma_prices_data = self._get_historical_data(start_date, end_date)
        close_df = self._get_close_df(last_ma_prices_data)
        ma_values_df = close_df.dropna().iloc[-ma:].mean()
        ma_pos_list = self._get_ma_position_list(last_session_price_dict, dict(ma_values_df))
        price_ma_df = self._create_ma_position_df(ma_pos_list, last_session_price_dict)
        self.__save_price_ma_df(price_ma_df, ma)
        return price_ma_df

    def _get_last_session_day(self, curr_day=datetime.today()):
        last = curr_day - timedelta(days=1)
        # to skip weekend days
        if last.weekday() > 4:
            return self._get_last_session_day(last)
        else:
            return last.strftime('%Y-%m-%d')

    @staticmethod
    def _get_price_dict_from_hist_data(data):
        ''' Get the close prices from historical data (from yahoo).
            Returns a dictionary with ticker as key and close price as value. '''
        price_dict = {}
        for ticker, hist_data in data.items():
            close_price = hist_data['prices'][0]['close']
            price_dict.update({ticker: close_price})
        return price_dict

In [81]:
index_single = 'wig20'
wig20_sf = SignalFinder(index_single)
wig20_sf.get_tickers_list()

['ALE.WA',
 'ACP.WA',
 'CCC.WA',
 'CDR.WA',
 'CPS.WA',
 'DNP.WA',
 'JSW.WA',
 'KGH.WA',
 'LTS.WA',
 'LPP.WA',
 'MBK.WA',
 'OPL.WA',
 'PEO.WA',
 'PCO.WA',
 'PGE.WA',
 'PGN.WA',
 'PKN.WA',
 'PKO.WA',
 'PZU.WA',
 'SPL.WA']

In [82]:
wig20_sf._check_components_validity()

True

In [61]:
# wig20_sf.get_comp_ath_list()

[]

In [60]:
wig20_sf.get_new_ath_df(save=True)

ERROR! Session/line number was not unique in database. History logging moved to new session 237
ALE.WA       94.639999
ACP.WA      176.765228
CCC.WA      309.000000
CDR.WA      460.799988
CPS.WA       37.860001
DNP.WA      374.799988
JSW.WA      141.500000
KGH.WA      223.800003
LTS.WA       99.459999
LPP.WA    18770.000000
MBK.WA      600.000000
OPL.WA       40.000000
PEO.WA      271.700012
PCO.WA       56.220001
PGE.WA       25.700001
PGN.WA        7.660000
PKN.WA      134.000000
PKO.WA       54.857697
PZU.WA       51.099998
SPL.WA      442.000000
dtype: float64


,ATH
Ticker,
ALE.WA,94.639999
ACP.WA,176.765228
CCC.WA,309.000000
CDR.WA,460.799988
CPS.WA,37.860001
DNP.WA,374.799988
JSW.WA,141.500000
KGH.WA,223.800003
LTS.WA,99.459999


In [10]:
# wig20_sf.save_tickers_list()

In [11]:
wig20_sf._check_components_validity()

True

In [69]:
wig20_sf.get_comp_x_years_max(10)

[]

In [83]:
wig20_sf.get_ma_position_change(50)

It seems there is no changes. Is it weekend, holiday or market was so boring today?


In [70]:
wig20_sf.get_initial_price_ma_position(50)

ERROR! Session/line number was not unique in database. History logging moved to new session 238


,Ticker,Price,MA,Position
0,ALE.WA,23.000000,28.35250,below
1,ACP.WA,76.800003,77.28700,below
2,CCC.WA,47.590000,54.32420,below
3,CDR.WA,120.680000,154.04600,below
4,CPS.WA,22.280001,26.57760,below
5,DNP.WA,301.700012,308.67800,below
6,JSW.WA,68.000000,69.76360,below
7,KGH.WA,119.199997,161.16700,below
8,LTS.WA,68.440002,63.17600,above
9,LPP.WA,9850.000000,10037.10000,below


In [18]:
index_all = 'wig20+mwig40+swig80'

In [19]:
wig20_40_80_sf = SignalFinder(index_all)

In [20]:
wig20_40_80_sf.get_tickers_list()

['ALE.WA',
 'ACP.WA',
 'CCC.WA',
 'CDR.WA',
 'CPS.WA',
 'DNP.WA',
 'JSW.WA',
 'KGH.WA',
 'LTS.WA',
 'LPP.WA',
 'MBK.WA',
 'OPL.WA',
 'PEO.WA',
 'PCO.WA',
 'PGE.WA',
 'PGN.WA',
 'PKN.WA',
 'PKO.WA',
 'PZU.WA',
 'SPL.WA',
 '11B.WA',
 'ALR.WA',
 'EAT.WA',
 'ASB.WA',
 'ASE.WA',
 'BFT.WA',
 'BML.WA',
 'BDX.WA',
 'CIE.WA',
 'CLN.WA',
 'CMR.WA',
 'DAT.WA',
 'DVL.WA',
 'DOM.WA',
 'ENA.WA',
 'EUR.WA',
 'FMF.WA',
 'GPW.WA',
 'ATT.WA',
 'BHW.WA',
 'HUG.WA',
 'ING.WA',
 'CAR.WA',
 'KER.WA',
 'KTY.WA',
 'KRU.WA',
 'LVC.WA',
 'MAB.WA',
 'MRC.WA',
 'MIL.WA',
 'MBR.WA',
 'NEU.WA',
 'PEP.WA',
 'PKP.WA',
 'PLW.WA',
 'SLV.WA',
 'TPE.WA',
 'TEN.WA',
 'WPL.WA',
 'XTB.WA',
 'ABE.WA',
 'ACG.WA',
 'ACT.WA',
 'AGO.WA',
 'AML.WA',
 'AMB.WA',
 'AMC.WA',
 'ANR.WA',
 'APT.WA',
 'ATC.WA',
 'ABS.WA',
 'AST.WA',
 '1AT.WA',
 'ATG.WA',
 'APR.WA',
 'BIO.WA',
 'BNP.WA',
 'LWB.WA',
 'BRS.WA',
 'BOS.WA',
 'BOW.WA',
 'BMC.WA',
 'CTX.WA',
 'CAV.WA',
 'CIG.WA',
 'CLE.WA',
 'COG.WA',
 'CMP.WA',
 'CRJ.WA',
 'DCR.WA',
 'ECH.WA',

In [21]:
wig20_40_80_sf._check_components_validity()

True

In [7]:
# wig20_40_80_sf._read_csv_data('ath')

In [24]:
wig20_40_80_sf.get_comp_ath_list()

ATH_Data\ATH_wig20_mwig40_swig80_2022-04-28.csv


[]

In [23]:
wig20_40_80_sf.get_comp_x_years_max(10)

[]

In [22]:
wig20_40_80_sf.get_ma_position_change(50)

PGE.WA BEAT the MA
DAT.WA BEAT the MA
ENA.WA BEAT the MA
XTB.WA BEAT the MA
ABE.WA BEAT the MA
LWB.WA BEAT the MA
GRN.WA BEAT the MA
RFK.WA BEAT the MA
VRG.WA LOST the MA
WWL.WA BEAT the MA
